In [1]:
import os
from sklearn.model_selection import KFold
import numpy as np

class CFG:
    class data:
        batch_size = 16
        lr = 1e-3 #5e-5
        epochs = 30
        epsilon = 1e-8
        MAX_LEN = 128 #max sentence length
        seed_val = 42 #random seed
        k_folds = 10
        hidden_size = 768 #hidden layer size (embedding size) for feedforward net
        PATH = "./"
        
        #defaults for CNN
        dropout = 0.4
        Ks = [1,2,3,4]
        kernel_num = 32 #number of filters for each conv layer

In [2]:
import torch
from torch import nn
import torch.nn.functional as F

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1660 Ti


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
import pandas as pd

df = pd.read_csv('ETHOS_binary_pre.csv')

print('Number of training sentences: {:,}\n'.format(df.shape[0]))

sentences = df.text.values
labels = df.label.values

df.head(10)

Number of training sentences: 1,054



,text,label
0,You should know women's sports are a joke,1
1,You look like Sloth with deeper Down’s syndrome,1
2,You look like Russian and speak like Indian. B...,1
3,"Women deserve to be abused, I guess.",1
4,Women are made for making babies and cooking d...,1
5,why haven't you faggots taken the bi-pill yet,1
6,Why don’t you go beck your god to murder you? ...,1
7,why does that african american monkey hold gun...,1
8,Why does Bulgaria exist as a country? It's one...,1
9,Why do negroes tear down their own neighborhoo...,1


In [5]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

#Load the Bert model.
print('Loading BERT Model...')
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
 
# Tell pytorch to run this model on the GPU.
model.cuda()

C:\Users\april\anaconda3\envs\hs\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading BERT tokenizer...
Loading BERT Model...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [6]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                   )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  You should know women's sports are a joke
Token IDs: [101, 2017, 2323, 2113, 2308, 1005, 1055, 2998, 2024, 1037, 8257, 102]


In [7]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Max sentence length:  166


In [8]:
# We'll borrow the `pad_sequences` utility function to do this.
from keras_preprocessing.sequence import pad_sequences

print('\nPadding/truncating all sentences to %d values...' % CFG.data.MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=CFG.data.MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 128 values...

Padding token: "[PAD]", ID: 0

Done.


In [9]:
# Create attention masks
attention_masks = []

# For each sentence...
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

In [10]:
#one-hot encode labels
def one_hot(categorical_labels):
    num_categories = 2
    labels = np.zeros((categorical_labels.shape[0], num_categories))
    labels[:,0] = (categorical_labels == 0).astype(int)
    labels[:,1] = (categorical_labels == 1).astype(int)
    return labels

labels = one_hot(labels)

In [11]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup

# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
# train_inputs = torch.tensor(train_inputs)
# validation_inputs = torch.tensor(validation_inputs)

# train_labels = torch.tensor(train_labels)
# validation_labels = torch.tensor(validation_labels)

# train_masks = torch.tensor(train_masks)
# validation_masks = torch.tensor(validation_masks)

input_ids = torch.tensor(input_ids)
labels = torch.tensor(labels)
attention_masks = torch.tensor(attention_masks)

dataset = TensorDataset(input_ids, labels, attention_masks)

In [12]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
# def flat_accuracy(preds, labels):
#     labels_flat = labels.flatten()
#     return np.sum(pred_flat == labels_flat) / len(labels_flat)

def flat_accuracy(preds, labels):
    y_hat = np.argmax(preds, axis=1).flatten()
    y = labels[:,1].flatten()
      
    #find tp, tn, fp, fn
    tp = ((y==1) & (y_hat==1)).sum()
    tn = ((y==0) & (y_hat==0)).sum()
    fp = ((y==0) & (y_hat==1)).sum()
    fn = ((y==1) & (y_hat==0)).sum()
    total = len(labels)
    return tp, tn, fp, fn, total
    
def get_accuracy(tp, tn, fp, fn, total):
    epsilon = CFG.data.epsilon

    def accuracy():
      return (tp + tn)/(total + epsilon) 

    def precision():
      return tp/(tp + fp + epsilon)
    
    def recall():
      return tp/(tp + fn + epsilon)

    def f1():
      p = precision()
      r = recall()
      return 2*p*r/(p + r + epsilon)

    return {
        'accuracy': accuracy(),
        'precision': precision(),
        'recall': recall(),
        'f1-score': f1()
    }

In [13]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

for layer in range(12):
    model.bert.encoder.layer[layer].output.LayerNorm.register_forward_hook(get_activation(layer))

In [14]:
input_ids_ = input_ids[:5].to(device)
input_mask_ = attention_masks[:5].to(device)
input_labels_ = labels[:5].to(device)

In [15]:
input_ids_.shape

torch.Size([5, 128])

In [16]:
outputs = model(input_ids_, 
                token_type_ids=None, 
                attention_mask=input_mask_)

In [17]:
import copy
a1 = copy.deepcopy(activation)

In [18]:
print(a1[11].unsqueeze(1).shape)
a1[11].mean()
# a1[11].mean(dim=1).shape
# a1[11][:,0,:].shape
# tt = activation[0][:,0,:].unsqueeze(dim=1)
# for layer in range(1, 12):
#     tt = torch.hstack((tt, activation[layer][:,0,:].unsqueeze(dim=1)))
# tt.shape
# tt2 = torch.stack((tt, tt))
# torch.stack((tt2,tt))
# tt.shape

torch.Size([5, 1, 128, 768])


tensor(-0.0082, device='cuda:0')

In [19]:
class Net(nn.Module):
    def __init__(self, input_shape):
        '''
        input_shape -> tuple (n,c,h,w)
        n = batch size
        c = num channels
        h = height
        w = width(768)
        '''
        super().__init__()
        #set default values for conv net
        dropout = CFG.data.dropout
        Ks = CFG.data.Ks
        Co = CFG.data.kernel_num #number of filters for each conv layer
        D = input_shape[3] 
        
        self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, (K, D)) for K in Ks])
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(len(Ks) * Co, 2)
        self.sigmoid = nn.Sigmoid()   
        
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.gelu = nn.GELU()
        self.selu = nn.SELU()
        self.leakyrelu = nn.LeakyReLU()     
    def forward(self, x):
        
        x = [self.leakyrelu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, W), ...]*len(Ks)

        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)
        x = torch.cat(x, 1)
        
        x = self.dropout(x)  # (N, len(Ks)*Co)
        
        x = self.leakyrelu(x)
        x = self.fc1(x)
        return x


In [20]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

for layer in range(12):
    model.bert.encoder.layer[layer].output.LayerNorm.register_forward_hook(get_activation(layer))

def prepare_data(inputs, masks, labels):
    inputs = inputs.to(device)
    masks = masks.to(device)
    labels = labels.to(device)
    outputs = model(inputs, 
                token_type_ids=None, 
                attention_mask=masks)
    inputs = activation[11].unsqueeze(1)
#     inputs = activation[11].mean(dim=1) #exp1: taking mean of all tokens in last layer
    
#     inputs = activation[0][:,0,:].unsqueeze(dim=1)
#     for layer in range(1, 12):
#         inputs = torch.hstack((inputs, activation[layer][:,0,:].unsqueeze(dim=1)))
#     inputs = inputs.unsqueeze(dim=1)
#     inputs = torch.reshape(activation[11], (activation[11].shape[0], -1))
#     cnt = 1e-8
#     inputs = torch.zeros_like(activation[0][:,0,:])
#     for i in range(9,12):
#       cnt += 1
#       inputs += activation[i][:,0,:]
#     inputs/=cnt
    
    return [inputs, labels]

In [21]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [22]:
batch_size = CFG.data.batch_size
filepath = CFG.data.PATH
epochs = CFG.data.epochs

def train(train_data, validation_data, fold_id, exp_no):
    '''
      Data: (list) -> [inputs, masks, labels]
    '''
    data_copy = []
    for entry in train_data:
        data_copy.append(entry[:2])
        
    inputs_, _ = prepare_data(*data_copy)
    net = Net(inputs_.shape)
    net.cuda()

    # Create the DataLoader for our training set.
    train_data = TensorDataset(*train_data)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Create the DataLoader for our validation set.
    validation_data = TensorDataset(*validation_data)
    validation_sampler = SequentialSampler(validation_data)
    validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

    # Function to validate model
    def validate_model(validation_dataloader):
        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        net.eval()

        # Tracking variables 
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0

        tp, tn, fp, fn, total = 0, 0, 0, 0, 0
        # Evaluate data for one epoch
        for batch in validation_dataloader:
            
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            
            # Telling the model not to compute or store gradients, saving memory and
            # speeding up validation
            with torch.no_grad():        

                # Forward pass, calculate logit predictions.
                # This will return the logits rather than the loss because we have
                # not provided labels.
                # token_type_ids is the same as the "segment ids", which 
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                # The documentation for this `model` function is here: 
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                b_inputs, b_labels = prepare_data(b_input_ids, b_input_mask, b_labels)

                # Get the "logits" output by the model. The "logits" are the output
                # values prior to applying an activation function like the softmax.
                logits = net(b_inputs)
                

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            
            # Calculate the accuracy for this batch of test sentences.
            tp_, tn_, fp_, fn_, tot_ = flat_accuracy(logits, label_ids)
            
            tp+=tp_
            tn+=tn_
            fp+=fp_
            fn+=fn_
            total+=tot_

            # Track the number of batches
            nb_eval_steps += 1

        # Report the final accuracy for this validation run.
        acc_metrics = get_accuracy(tp, tn, fp, fn, total)
        for k, v in acc_metrics.items():
          print("{} : {:.5f},".format(k,v), end=" ")

        print("\nValidation took: {:}".format(format_time(time.time() - t0)))
        return acc_metrics
    

    best_metrics = {}
    best_f1_so_far = 0.0
    
    # Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
    optimizer = AdamW(net.parameters(),
                      lr = CFG.data.lr, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                      eps = CFG.data.epsilon
                    )
    
    #define loss function
    criterion = nn.CrossEntropyLoss()

    # Total number of training steps is number of batches * number of epochs.
    total_steps = len(train_dataloader) * epochs

    # Create the learning rate scheduler.
#     scheduler = get_linear_schedule_with_warmup(optimizer, 
#                                                 num_warmup_steps = 0, # Default value in run_glue.py
#                                                 num_training_steps = total_steps)
    
    print('Current Best f1-score: {:.5}'.format(best_f1_so_far))

    # For each epoch...
    for epoch_i in range(0, epochs):
        
        # ========================================
        #               Training
        # ========================================
        
        # Perform one full pass over the training set.

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        
        # For each batch of training data...
        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_loss = 1

        # Put the model into training mode. Don't be mislead--the call to 
        # `train` just changes the *mode*, it doesn't *perform* the training.
        # `dropout` and `batchnorm` layers behave differently during training
        # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
        net.train()

        print('Training...')
        for step, batch in enumerate(train_dataloader):

            # Progress update every 40 batches.
#             if step % 40 == 0 and not step == 0:
#                 # Calculate elapsed time in minutes.
#                 elapsed = format_time(time.time() - t0)
                
#                 # Report progress.
#                 print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using the 
            # `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            b_inputs, b_labels = prepare_data(b_input_ids, b_input_mask, b_labels)

            # Always clear any previously calculated gradients before performing a
            # backward pass. PyTorch doesn't do this automatically because 
            # accumulating the gradients is "convenient while training RNNs". 
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            net.zero_grad()        

            # Perform a forward pass (evaluate the model on this training batch).
            # This will return the loss (rather than the model output) because we
            # have provided the `labels`.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
    
            outputs = net(b_inputs)
                
            # The call to `model` always returns a tuple, so we need to pull the 
            # loss value out of the tuple.
            loss = criterion(outputs, b_labels)

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            total_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(net.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
#             scheduler.step()

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(train_dataloader)            
        
        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(format_time(time.time() - t0)))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        print("")
        print("Running Validation...")

        accuracy_metrics = validate_model(validation_dataloader)
        current_f1 = accuracy_metrics['f1-score']
        if current_f1>best_f1_so_far:
            print('f1 score improved, old f1 = {:.5}, new f1 = {:.5}'.format(best_f1_so_far, current_f1))
            best_f1_so_far = current_f1
            best_metrics = accuracy_metrics
            print('Saving new weights...')
            torch.save(net.state_dict(), filepath + f'/results/net_exp{exp_no}_{fold_id}.pth')

        elif current_f1 == best_f1_so_far:
            print('f1 score did not change')
            b_acc = 0.0
            if 'accuracy' in best_metrics:
                b_acc = best_metrics['accuracy']
            cur_acc = accuracy_metrics['accuracy']
            if cur_acc>=b_acc:
                print('accuracy score improved, old acc = {:.5}, new acc = {:.5}'.format(b_acc, cur_acc))
                best_metrics = accuracy_metrics
                print('Saving new weights...')
                torch.save(net.state_dict(), filepath + f'/results/net_exp{exp_no}_{fold_id}.pth')

    return best_metrics

In [23]:
val_inputs_dict = {}

In [24]:
kfold = KFold(n_splits=CFG.data.k_folds, shuffle=True)
best_f1_scores = {}
scores = {'accuracy' : [], 'precision' : [], 'recall' : [], 'f1-score' : []}
model.train(False)

for fold_id, (train_ids, val_ids) in enumerate(kfold.split(dataset)):

    print('------------------ Fold {} ------------------'.format(fold_id))

    train_inputs, train_masks, train_labels = input_ids[train_ids], attention_masks[train_ids], labels[train_ids]
    validation_inputs, validation_masks, validation_labels = input_ids[val_ids], attention_masks[val_ids], labels[val_ids]
    train_data = [train_inputs, train_masks, train_labels]
    validation_data = [validation_inputs, validation_masks, validation_labels]
    val_inputs_dict[fold_id] = val_ids
    print(f"Fold {fold_id} completed.....")
    
    #Model Training
    best_f1_scores[fold_id] = train(train_data, validation_data, fold_id, 1)
    
    for key in best_f1_scores[fold_id].keys():
        scores[key].append(best_f1_scores[fold_id][key])
    
    for k, v in scores.items():
        print('{}: {:.5f}, '.format(k, sum(v)/len(v)), end = "")
    print("")
print("")
print("Training complete!")

------------------ Fold 0 ------------------
Fold 0 completed.....
Current Best f1-score: 0.0

======== Epoch 1 / 30 ========
Training...


C:\Users\april\anaconda3\envs\hs\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



  Average training loss: 0.65
  Training epoch took: 0:00:12

Running Validation...
accuracy : 0.68868, precision : 0.86667, recall : 0.29545, f1-score : 0.44068, 
Validation took: 0:00:01
f1 score improved, old f1 = 0.0, new f1 = 0.44068
Saving new weights...

======== Epoch 2 / 30 ========
Training...

  Average training loss: 0.46
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.82075, precision : 0.82051, recall : 0.72727, f1-score : 0.77108, 
Validation took: 0:00:01
f1 score improved, old f1 = 0.44068, new f1 = 0.77108
Saving new weights...

======== Epoch 3 / 30 ========
Training...

  Average training loss: 0.35
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.83962, precision : 0.80000, recall : 0.81818, f1-score : 0.80899, 
Validation took: 0:00:01
f1 score improved, old f1 = 0.77108, new f1 = 0.80899
Saving new weights...

======== Epoch 4 / 30 ========
Training...

  Average training loss: 0.26
  Training epoch took: 0:00:11

Running Val


  Average training loss: 0.37
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.76415, precision : 0.68182, recall : 0.73171, f1-score : 0.70588, 
Validation took: 0:00:01
f1 score improved, old f1 = 0.69725, new f1 = 0.70588
Saving new weights...

======== Epoch 4 / 30 ========
Training...

  Average training loss: 0.27
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.76415, precision : 0.64286, recall : 0.87805, f1-score : 0.74227, 
Validation took: 0:00:01
f1 score improved, old f1 = 0.70588, new f1 = 0.74227
Saving new weights...

======== Epoch 5 / 30 ========
Training...

  Average training loss: 0.21
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.79245, precision : 0.71111, recall : 0.78049, f1-score : 0.74419, 
Validation took: 0:00:01
f1 score improved, old f1 = 0.74227, new f1 = 0.74419
Saving new weights...

======== Epoch 6 / 30 ========
Training...

  Average training loss: 0.13
  Training epoch took: 0:00:10

Running


  Average training loss: 0.17
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.77358, precision : 0.85714, recall : 0.61224, f1-score : 0.71429, 
Validation took: 0:00:01

======== Epoch 6 / 30 ========
Training...

  Average training loss: 0.13
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.83962, precision : 0.82000, recall : 0.83673, f1-score : 0.82828, 
Validation took: 0:00:01
f1 score improved, old f1 = 0.81633, new f1 = 0.82828
Saving new weights...

======== Epoch 7 / 30 ========
Training...

  Average training loss: 0.09
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.78302, precision : 0.84211, recall : 0.65306, f1-score : 0.73563, 
Validation took: 0:00:01

======== Epoch 8 / 30 ========
Training...

  Average training loss: 0.07
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.81132, precision : 0.83721, recall : 0.73469, f1-score : 0.78261, 
Validation took: 0:00:01

======== Epoch 9 / 30 ========


...

accuracy : 0.77143, precision : 0.69048, recall : 0.72500, f1-score : 0.70732, 
Validation took: 0:00:01

======== Epoch 13 / 30 ========
Training...

  Average training loss: 0.03
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.79048, precision : 0.70455, recall : 0.77500, f1-score : 0.73810, 
Validation took: 0:00:01

======== Epoch 14 / 30 ========
Training...

  Average training loss: 0.03
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.73333, precision : 0.62500, recall : 0.75000, f1-score : 0.68182, 
Validation took: 0:00:01

======== Epoch 15 / 30 ========
Training...

  Average training loss: 0.02
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.76190, precision : 0.64151, recall : 0.85000, f1-score : 0.73118, 
Validation took: 0:00:01

======== Epoch 16 / 30 ========
Training...

  Average training loss: 0.02
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.77143, precision : 0.66667, recall : 0.80000, f


  Average training loss: 0.02
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.69524, precision : 0.77778, recall : 0.53846, f1-score : 0.63636, 
Validation took: 0:00:01

======== Epoch 16 / 30 ========
Training...

  Average training loss: 0.03
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.80952, precision : 0.78571, recall : 0.84615, f1-score : 0.81481, 
Validation took: 0:00:01

======== Epoch 17 / 30 ========
Training...

  Average training loss: 0.02
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.80000, precision : 0.78182, recall : 0.82692, f1-score : 0.80374, 
Validation took: 0:00:01

======== Epoch 18 / 30 ========
Training...

  Average training loss: 0.02
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.72381, precision : 0.74468, recall : 0.67308, f1-score : 0.70707, 
Validation took: 0:00:01

======== Epoch 19 / 30 ========
Training...

  Average training loss: 0.02
  Training epoch took: 0:00:1


  Average training loss: 0.02
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.73333, precision : 0.81481, recall : 0.48889, f1-score : 0.61111, 
Validation took: 0:00:01

======== Epoch 19 / 30 ========
Training...

  Average training loss: 0.03
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.78095, precision : 0.77500, recall : 0.68889, f1-score : 0.72941, 
Validation took: 0:00:01

======== Epoch 20 / 30 ========
Training...

  Average training loss: 0.02
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.74286, precision : 0.80000, recall : 0.53333, f1-score : 0.64000, 
Validation took: 0:00:01

======== Epoch 21 / 30 ========
Training...

  Average training loss: 0.02
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.80000, precision : 0.80000, recall : 0.71111, f1-score : 0.75294, 
Validation took: 0:00:01

======== Epoch 22 / 30 ========
Training...

  Average training loss: 0.02
  Training epoch took: 0:00:1


  Average training loss: 0.02
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.80000, precision : 0.81818, recall : 0.73469, f1-score : 0.77419, 
Validation took: 0:00:01

======== Epoch 22 / 30 ========
Training...

  Average training loss: 0.02
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.82857, precision : 0.84444, recall : 0.77551, f1-score : 0.80851, 
Validation took: 0:00:01

======== Epoch 23 / 30 ========
Training...

  Average training loss: 0.03
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.80952, precision : 0.89189, recall : 0.67347, f1-score : 0.76744, 
Validation took: 0:00:01

======== Epoch 24 / 30 ========
Training...

  Average training loss: 0.03
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.81905, precision : 0.87500, recall : 0.71429, f1-score : 0.78652, 
Validation took: 0:00:01

======== Epoch 25 / 30 ========
Training...

  Average training loss: 0.02
  Training epoch took: 0:00:1


  Average training loss: 0.03
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.74286, precision : 0.65116, recall : 0.70000, f1-score : 0.67470, 
Validation took: 0:00:01

======== Epoch 25 / 30 ========
Training...

  Average training loss: 0.02
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.77143, precision : 0.69048, recall : 0.72500, f1-score : 0.70732, 
Validation took: 0:00:01

======== Epoch 26 / 30 ========
Training...

  Average training loss: 0.02
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.76190, precision : 0.68293, recall : 0.70000, f1-score : 0.69136, 
Validation took: 0:00:01

======== Epoch 27 / 30 ========
Training...

  Average training loss: 0.02
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.79048, precision : 0.70455, recall : 0.77500, f1-score : 0.73810, 
Validation took: 0:00:01

======== Epoch 28 / 30 ========
Training...

  Average training loss: 0.02
  Training epoch took: 0:00:1

In [25]:
for fold_id in range(10):
    print(f"{fold_id}: ", end = " ")
    for k, v in best_f1_scores[fold_id].items():
        print("{} : {:.5f},".format(k,v), end=" ")
    print("")

0:  accuracy : 0.85849, precision : 0.85366, recall : 0.79545, f1-score : 0.82353, 
1:  accuracy : 0.82075, precision : 0.73913, recall : 0.82927, f1-score : 0.78161, 
2:  accuracy : 0.84906, precision : 0.77966, recall : 0.93878, f1-score : 0.85185, 
3:  accuracy : 0.72642, precision : 0.69231, recall : 0.73469, f1-score : 0.71287, 
4:  accuracy : 0.83810, precision : 0.85714, recall : 0.76596, f1-score : 0.80899, 
5:  accuracy : 0.82857, precision : 0.75000, recall : 0.82500, f1-score : 0.78571, 
6:  accuracy : 0.82857, precision : 0.75758, recall : 0.96154, f1-score : 0.84746, 
7:  accuracy : 0.82857, precision : 0.77551, recall : 0.84444, f1-score : 0.80851, 
8:  accuracy : 0.86667, precision : 0.87234, recall : 0.83673, f1-score : 0.85417, 
9:  accuracy : 0.79048, precision : 0.68750, recall : 0.82500, f1-score : 0.75000, 


In [26]:
for k, v in scores.items():
    print("{} : {:.5f},".format(k,sum(v)/len(v)), end=" ")
print("")

accuracy : 0.82357, precision : 0.77648, recall : 0.83569, f1-score : 0.80247, 
